# ClimAgent quick start

In [1]:
from climagent.agent.climagent import ClimAgent
from langchain_core.messages import HumanMessage
from langchain_openai import AzureChatOpenAI
import os
from dotenv import load_dotenv
import xarray as xr

load_dotenv("../credentials.env")

import warnings
warnings.filterwarnings("ignore")

In [2]:
# Initialize LLM
llm = AzureChatOpenAI(
    deployment_name=os.getenv("GPT_DEPLOYMENT_NAME"),
    openai_api_version=os.getenv("OPENAI_API_VERSION"),
    openai_api_key=os.getenv("OPENAI_API_KEY"),
    azure_endpoint=os.getenv("AZURE_ENDPOINT"),
    temperature=0,
)

In [ ]:
dataset_path = "../example_datasets/cckp/store/CMIP6_timeseries.zarr"
dataset = xr.open_zarr(dataset_path, decode_timedelta=True)
dataset

In [4]:
agent = ClimAgent(dataset, llm)
# agent.graph

In [5]:
query: str = (
    "I want to assess whether Rome is expected to experience heat waves conditions hazardous to the population in the period 2040-2060.\n"
    # "Proceed as follows:\n"
    # "1. Select the data for Rome.\n"
    # "2. Select the data for the period 2040-2060.\n"
    # "3. Select the data for the variables that you think are most relevant to assess the heat waves conditions.\n"
    # "4. Aggregate percentiles using mean. "
    # "5. Do not aggregate over the scenario dimension.\n"
    # "6. Describe the results you obtain and answer the question.\n"
)

messages = [HumanMessage(query)]
response = agent.run(messages)

In [6]:
print(response['messages'][-1].content)

The analysis of heat wave conditions in Rome for the period 2040-2060 has been successfully set up. Here are the key findings and next steps:

### Key Findings:
1. **Geographic Focus**: The dataset has been filtered to focus on Rome, specifically at the coordinates (latitude: 41.0, longitude: 12.0).
2. **Time Period**: The analysis is restricted to the years 2040 to 2060.
3. **Selected Variables**:
   - **`tasmax`**: Daily maximum near-surface air temperature.
   - **`hi`**: Heat index, which combines temperature and humidity to reflect perceived temperature.

### Next Steps:
1. **Heat Wave Definition**: Define what constitutes a heat wave for this analysis (e.g., days with `tasmax` exceeding 35°C or 40°C).
2. **Calculate Heat Wave Metrics**:
   - Count the number of days exceeding the defined thresholds.
   - Analyze the duration and frequency of these heat wave events.
3. **Assess Population Hazard Levels**: Evaluate the potential impact of these heat waves on the population based on

In [7]:
agent.dataset_state.history

['Subset on lat(41.0)',
 'Subset on lon(12.0)',
 'Subset on time(2040-01-01:2060-12-31)',
 "Selected variables:['tasmax', 'hi'])"]

In [8]:
agent.dataset_state.dataset

<xarray.Dataset> Size: 2kB
Dimensions:     (percentile: 3, scenario: 3, time: 21)
Coordinates:
    lat         float64 8B 41.0
    lon         float64 8B 12.0
  * percentile  (percentile) <U6 72B 'median' 'p10' 'p90'
  * scenario    (scenario) <U6 72B 'ssp126' 'ssp245' 'ssp585'
  * time        (time) datetime64[ns] 168B 2040-07-01 2041-07-01 ... 2060-07-01
Data variables:
    tasmax      (percentile, scenario, time) float32 756B dask.array<chunksize=(3, 3, 21), meta=np.ndarray>
    hi          (percentile, scenario, time) float32 756B dask.array<chunksize=(3, 3, 21), meta=np.ndarray>
Attributes: (12/43)
    Conventions:           CF-1.7 CMIP-6.2
    activity_id:           ScenarioMIP
    climate_data_id:       CMIP6
    cmip6-ng:              Some daily data has been acquired through the CMIP...
    cmor_version:          3.4.0
    creation_date:         2021-11-05T01:00:00Z
    ...                    ...
    wb_scenario_label:     ssp126
    wb_statistic_label:    mean
    wb_truncation_label:   2
    wb_type_label:         timeseries
    wb_variable_label:     tas
    wb_variant_label:      all